In [51]:
import pandas as pd

1. 데이터 불러오기

In [52]:
# CSV 파일 불러오기
google_df = pd.read_csv('../데이터/구글_트렌드/구글_트렌드_병합_데이터.csv')
incident_df = pd.read_csv('../데이터/이벤트 스터디 결과/사건별_정당성_검정결과_tmp.csv')

2. 날짜 전처리 (YYYY-MM 형태로 변환)

In [53]:
# 사건 날짜를 datetime 형식으로 변환 후 '년월' 컬럼 생성
incident_df['사건 날짜'] = pd.to_datetime(incident_df['사건 날짜'], errors='coerce')
incident_df['년월'] = incident_df['사건 날짜'].dt.to_period('M').astype(str)

# 구글 트렌드의 월도 동일하게 변환
google_df['월'] = pd.to_datetime(google_df['월'], format='%Y-%m')
google_df['년월'] = google_df['월'].dt.to_period('M').astype(str)

3. 관심도 값 병합 함수 정의 및 적용

In [54]:
def get_interest(row):
    date = row['년월']
    name = row['연예인 이름']
    try:
        value = google_df.loc[google_df['년월'] == date, name].values
        if len(value) > 0:
            return value[0]
    except KeyError:
        return None
    return None

incident_df['관심도'] = incident_df.apply(get_interest, axis=1)


In [55]:
incident_df.head()

,소속사,그룹,연예인 이름,사건 내용,사건 날짜,매핑된 주가 날짜,CAR,t통계량,p값,시가총액,년월,관심도
0,HYBE,방탄소년단,지민,"2022년 4월 25일, 언론사 비즈한국의 취재결과 건강보험료 체납으로 보유하고 있...",2022-04-25,2022-04-25,0.042487,-27.921501,2.089598e-93,1.035900e+13,2022-04,0
1,HYBE,방탄소년단,정국,"2024년 1월 6일, 정국이 입대 후 신병 교육을 받는 동안 발생했다. 탈취범은 ...",2024-01-06,2024-01-08,0.002567,-2.184221,2.923824e-02,1.045470e+13,2024-01,0
2,HYBE,엔하이픈,정원,"2021년 11월 18일 오후 5시 , ENHYPEN 은 D-1 이라는 제목으로 V...",2021-11-18,2021-11-18,-0.049700,28.198598,4.936863e-82,1.610850e+13,2021-11,0
3,HYBE,엔하이픈,정원,2021년 11월 19일 사과문을 게시했다. [ 펼치기 · 접기 ] 엔진 여러분 안...,2021-11-19,2021-11-19,0.007088,-2.728296,6.787166e-03,1.563990e+13,2021-11,0
4,HYBE,엔하이픈,제이,"2023년 1월 10일, 멤버 성훈과 함께 진행한 위버스 라이브 도중 요즘 한국사 ...",2023-01-10,2023-01-10,-0.005359,4.277794,2.226093e-05,7.050750e+12,2023-01,<1


관심도 '<1' 처리 및 숫자형 변환

In [56]:
# '<1' 값을 0.5로, 나머지는 숫자형으로 변환
def clean_interest(val):
    if val == '<1':
        return 0.5
    try:
        return float(val)
    except:
        return None

incident_df['관심도'] = incident_df['관심도'].apply(clean_interest)

결과 확인

In [57]:
# 결과 미리보기
incident_df.head()

,소속사,그룹,연예인 이름,사건 내용,사건 날짜,매핑된 주가 날짜,CAR,t통계량,p값,시가총액,년월,관심도
0,HYBE,방탄소년단,지민,"2022년 4월 25일, 언론사 비즈한국의 취재결과 건강보험료 체납으로 보유하고 있...",2022-04-25,2022-04-25,0.042487,-27.921501,2.089598e-93,1.035900e+13,2022-04,0.0
1,HYBE,방탄소년단,정국,"2024년 1월 6일, 정국이 입대 후 신병 교육을 받는 동안 발생했다. 탈취범은 ...",2024-01-06,2024-01-08,0.002567,-2.184221,2.923824e-02,1.045470e+13,2024-01,0.0
2,HYBE,엔하이픈,정원,"2021년 11월 18일 오후 5시 , ENHYPEN 은 D-1 이라는 제목으로 V...",2021-11-18,2021-11-18,-0.049700,28.198598,4.936863e-82,1.610850e+13,2021-11,0.0
3,HYBE,엔하이픈,정원,2021년 11월 19일 사과문을 게시했다. [ 펼치기 · 접기 ] 엔진 여러분 안...,2021-11-19,2021-11-19,0.007088,-2.728296,6.787166e-03,1.563990e+13,2021-11,0.0
4,HYBE,엔하이픈,제이,"2023년 1월 10일, 멤버 성훈과 함께 진행한 위버스 라이브 도중 요즘 한국사 ...",2023-01-10,2023-01-10,-0.005359,4.277794,2.226093e-05,7.050750e+12,2023-01,0.5


In [58]:
incident_df = incident_df.drop(columns=['년월'])
incident_df.head()

,소속사,그룹,연예인 이름,사건 내용,사건 날짜,매핑된 주가 날짜,CAR,t통계량,p값,시가총액,관심도
0,HYBE,방탄소년단,지민,"2022년 4월 25일, 언론사 비즈한국의 취재결과 건강보험료 체납으로 보유하고 있...",2022-04-25,2022-04-25,0.042487,-27.921501,2.089598e-93,1.035900e+13,0.0
1,HYBE,방탄소년단,정국,"2024년 1월 6일, 정국이 입대 후 신병 교육을 받는 동안 발생했다. 탈취범은 ...",2024-01-06,2024-01-08,0.002567,-2.184221,2.923824e-02,1.045470e+13,0.0
2,HYBE,엔하이픈,정원,"2021년 11월 18일 오후 5시 , ENHYPEN 은 D-1 이라는 제목으로 V...",2021-11-18,2021-11-18,-0.049700,28.198598,4.936863e-82,1.610850e+13,0.0
3,HYBE,엔하이픈,정원,2021년 11월 19일 사과문을 게시했다. [ 펼치기 · 접기 ] 엔진 여러분 안...,2021-11-19,2021-11-19,0.007088,-2.728296,6.787166e-03,1.563990e+13,0.0
4,HYBE,엔하이픈,제이,"2023년 1월 10일, 멤버 성훈과 함께 진행한 위버스 라이브 도중 요즘 한국사 ...",2023-01-10,2023-01-10,-0.005359,4.277794,2.226093e-05,7.050750e+12,0.5


In [60]:
# CSV 저장
incident_df.to_csv('../데이터/이벤트 스터디 결과/사건별_정당성_검정결과_tmp.csv', index=False)